In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import os

import mysql.connector
from sqlalchemy import create_engine, text

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../src')
from sql_upload_manager import *

In [2]:
files = []

for archivo in os.listdir('../data'):
        
        if archivo.endswith('.csv'):
            
                file = os.path.join('../data', archivo)
                
                files.append((file.replace('.csv', '').replace('Anos', 'Años').replace('_', ' ').replace('../data\\','')))
                

In [5]:
key = open('password.txt','r').read()

In [6]:
cursor = conection_to_SQL('root',key,'localhost')

In [ ]:
cursor.execute('CREATE DATABASE IF NOT EXISTS VitalStatsES')

In [9]:
for file in files:
    if file == 'id':
        add_table(file,'primary','VitalStatsES'
                  )

['Años de vida saludable al nacer',
 'Años de vida saludable a los 65 anos',
 'Consumo diario de frutas yo verduras en adultos',
 'Consumo diario de frutas yo verduras en jovenes',
 'Consumo intensivo de alcohol al menos una vez al mes en los ultimos 12 meses en jovenes',
 'Deterioro cognitivo en 65 y mas anos',
 'Dificultad para la movilidad en 65 y mas anos',
 'Esperanza de vida al nacer',
 'Esperanza de vida a los 65 anos',
 'id',
 'Incidencia de accidentes de trabajo',
 'Incidencia de nuevos diagnosticos de VIH',
 'Incidencia de tuberculosis por 100 000 hab',
 'Indice de calidad de vida infantil relacionada con la salud',
 'Indice de dependencia',
 'Indice de dependencia en 65 y mas anos',
 'Indice de dependencia juvenil',
 'Ingresos medios por persona euros',
 'Limitacion para la realizacion de actividades relacionadas con el hogar en 65 y mas anos',
 'Numero de victimas mortales de accidentes de trafico',
 'Numero de victimas no mortales de accidente de trafico',
 'Porcentaje de 